# Facial Recognition App

Link Google Drive and download model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/Face Detection/'

Install and Import Dependancies

In [ ]:
!pip install mtcnn

In [ ]:
import tensorflow as tf
import sys
import random
sys.path.append(root_path)
import mtcnn_detector
from google.colab.patches import cv2_imshow

In [ ]:
class Engine:
	@staticmethod
	def turn_rgb(images):
		final_images = []
		for image in images:
			b, g, r = tf.split(image, 3, axis=-1)
			image = tf.concat([r, g, b], -1)
			final_images.append(image)

		return final_images

	@staticmethod
	def set_face(face):
		face = tf.image.resize(face, (112, 112), method="nearest")

		return (tf.cast(face, tf.float32) - 127.5) / 128.

	def __init__(self, model_path: str):
		self.model = tf.keras.models.load_model(model_path)
		self.data = {}

		self.cos_dis = lambda x, y: tf.norm(x-y)  # tf.keras.losses.CosineSimilarity()

		self.detector = mtcnn_detector.Engine()

	def get_output(self, images):
		return tf.nn.l2_normalize(self.model(images, training=False))

	def __call__(self, main_path1: str, main_path2: str, person_name: str, th: float = 1.0):

		# Load image, detect faces and get boxes around faces in 1st image
		image1 = self.detector.load_image(main_path1)
		faces1 = self.detector.get_faces_from_image(image1)
		boxes1 = self.detector.get_boxes_from_faces(faces1)
		if len(faces1) < 1:
				print("Cant continue, no faces found in image 1")
				return;

		# Load image, detect faces and get boxes around faces in 2nd image
		image2 = self.detector.load_image(main_path2)
		faces2 = self.detector.get_faces_from_image(image2)
		boxes2 = self.detector.get_boxes_from_faces(faces2)
		if len(faces2) < 1:
				print("Cant continue, no faces found in image 2")
				return;

		# Extract features from 1st image
		face_frames = self.turn_rgb(self.detector.take_faces_from_boxes(image1, boxes1))
		face_frames = tf.convert_to_tensor([self.set_face(n) for n in face_frames])
		output1 = self.get_output(face_frames)[0]

		# Extract features from 2nd image
		face_frames = self.turn_rgb(self.detector.take_faces_from_boxes(image2, boxes2))
		face_frames = tf.convert_to_tensor([self.set_face(n) for n in face_frames])
		output2 = self.get_output(face_frames)

		match_color = (0, 255, 0)
		non_match_color = (0, 0, 255)

		# iterate through found faces and highlight with color
		labels = []
		colors = []
		for output in output2:
			dist = self.cos_dis(output1, output)
			if dist < th:
				print("Found",person_name,"in group")
				labels.append(person_name)
				colors.append(match_color)
			else:
				labels.append("")
				colors.append(non_match_color)

		image1 = self.detector.draw_faces_and_labels_on_image(image1, boxes1, [person_name], color=match_color)
		image2 = self.detector.draw_faces_and_labels_on_image(image2, boxes2, labels, color=colors)
		cv2_imshow(image1)
		cv2_imshow(image2)

In [ ]:
e = Engine(root_path+"/model.h5")
e(root_path+"person.jpeg", root_path+"group4.jpeg", "Person", th=1.0)
